# CLASSIFICATION STEP

## Important Libraries

In [2]:
import pandas as pd
import re
import string
import seaborn as sns
from pyarabic.araby import strip_harakat
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk 
from string import punctuation
import heapq  

## Reading the datasets

In [3]:
df_train = pd.read_csv("../datasets/101121_news_source_HSEP_train.csv" )
df_test = pd.read_csv("../datasets/101121_news_source_HSEP_test.csv" )

df_train.head(5)

,ID,Subject,Content
0,0,سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف,<p>متابعة – مظفر إسماعيل</p><p> </p><p>نفى “ل...
1,1,ماذا يفعل القرنفل على الريق؟,<p>خبرني - القرنفل على الريق هي ذات الفوائد ال...
2,2,لن تتوقعها.. حيلة ذكية تجعل ثمرة الأفوكادو تنض...,<p>متابعة – علي معلا:</p><p>يحب الجميع تناول ف...
3,3,السعودية تستضيف اليابان ضمن 6 مباريات فى تصفيا...,تقام اليوم الخميس، 6 مواجهات ضمن مباريات الجول...
4,4,انتهاء التصويت.. صناديق الاقتراع العام تغلق في...,أعلنت المفوضية العليا للانتخابات في العراق عن ...


### Concatentating both dataframes, to do preprocessing

In [70]:
df = pd.concat([df_train, df_test], axis=0)
df = df.drop(columns=["ID"])

df = df.reset_index(drop=True)
print('Shape of Total DF : {}'.format(df.shape))
df.head()

Shape of Total DF : (12225, 2)


,Subject,Content
0,سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف,<p>متابعة – مظفر إسماعيل</p><p> </p><p>نفى “ل...
1,ماذا يفعل القرنفل على الريق؟,<p>خبرني - القرنفل على الريق هي ذات الفوائد ال...
2,لن تتوقعها.. حيلة ذكية تجعل ثمرة الأفوكادو تنض...,<p>متابعة – علي معلا:</p><p>يحب الجميع تناول ف...
3,السعودية تستضيف اليابان ضمن 6 مباريات فى تصفيا...,تقام اليوم الخميس، 6 مواجهات ضمن مباريات الجول...
4,انتهاء التصويت.. صناديق الاقتراع العام تغلق في...,أعلنت المفوضية العليا للانتخابات في العراق عن ...


## Processing the data

### Cleaning the data

#### Removing Punctuations, Tashkeel (harakat), spaces, english letters ...

In [72]:
arabic_punctuations = '''«»`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
punctuations_list

'«»`÷×؛<>_()*&^%][ـ،/:"؟.,\'{}~¦+|!”…“–ـ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

#### Function to clean text

In [73]:
def clean(text):
    output = re.sub(r'\s*[A-Za-z]+\b', ' ' , text) #Remove english letters
    output = strip_harakat(output) #Remove harakat  

    translator = str.maketrans(' ',' ', punctuations_list) #remove arabic and english punctuations
    output = output.translate(translator)

    output = re.sub('\w*\d\w*', ' ', output)# Remove numbers
    output = " ".join(output.split()) #remove extra spaces
    
    
    return output

In [74]:
print("Text before Cleaning: \n {}".format(df.iloc[0,1]))
print('\n',"*" * 100,'\n')
print("Text After Cleaning:\n {}".format(clean(df.iloc[0,1])))

Text before Cleaning: 
 <p>متابعة –  مظفر إسماعيل</p><p> </p><p>نفى “لويس سواريز” تعمده إرسال رسالة إلى “رونالد كومان”، خلال لقاء “أتلتيكو مدريد” وضيفه “برشلونة” في الدوري الإسباني، السبت.</p><p> </p><p>وبعدما سجل “سواريز” هدف “أتلتيكو”  الثاني، أشار المهاجم المخضرم بيده وكأنه يمسك هاتفاً ويتحدث إلى شخص ما. وهو ما ربطه المتابعون بمكالمة “كومان” إلى “سواريز” في صيف 2020. التي أبلغه خلالها بأنه خرج من خططه وأن عليه الرحيل عن “برشلونة” والبحث عن فريق آخر.</p><p> </p><p>وقال “سواريز” بعد اللقاء الذي ابتسم لأصحاب الأرض، إن “الرسالة التي أراد توجيهها لم تكن لكومان على الإطلاق”.</p><p> </p><p>وتابع بلهجة غامضة في تصريحات نشرتها صحيفة “ماركا” الرياضية الإسبانية: “كان ذلك للأشخاص الذين يعرفون أن لدي نفس الرقم. وأنني ما زلت أستخدم نفس الهاتف.. إن كنت تريد أن تعرف، كنت أمزح مع أطفالي أنني سأفعل ذلك”.</p><p> </p><p>كما علق “سواريز” على عدم احتفاله بإحراز هدف في شباك فريقه السابق، موضحاً: “كنت أعرف بالفعل أنني إذا سجلت فسأعتذر”.</p><p> </p><p>وأضاف: “لم أحتفل احتراماً وتعاطفاً لكوني من مشجعي برشلون

#### Concatenate Subject & Content and Applying clean function to the Dataframe

In [75]:
df_cleaned = df.copy()

## Cleaning
df_cleaned.Content = df_cleaned.Content.map(clean)
df_cleaned.Subject = df_cleaned.Subject.map(clean)

## Concatenating
df_cleaned["Text"] = df_cleaned.Subject + " " + df_cleaned.Content

df_cleaned.head(10)

,Subject,Content,Text
0,سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف,متابعة مظفر إسماعيل نفى لويس سواريز تعمده إرسا...,سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف ...
1,ماذا يفعل القرنفل على الريق,خبرني القرنفل على الريق هي ذات الفوائد التي يج...,ماذا يفعل القرنفل على الريق خبرني القرنفل على ...
2,لن تتوقعها حيلة ذكية تجعل ثمرة الأفوكادو تنضج ...,متابعة علي معلا يحب الجميع تناول فاكهة الأفوكا...,لن تتوقعها حيلة ذكية تجعل ثمرة الأفوكادو تنضج ...
3,السعودية تستضيف اليابان ضمن مباريات فى تصفيات ...,تقام اليوم الخميس مواجهات ضمن مباريات الجولة ا...,السعودية تستضيف اليابان ضمن مباريات فى تصفيات ...
4,انتهاء التصويت صناديق الاقتراع العام تغلق في ع...,أعلنت المفوضية العليا للانتخابات في العراق عن ...,انتهاء التصويت صناديق الاقتراع العام تغلق في ع...
5,إنريكى مواجهة إسبانيا ضد إيطاليا صعبة وسنلعب ب...,أكد لويس انريكي المدير الفني لمنتخب اسبانيا عل...,إنريكى مواجهة إسبانيا ضد إيطاليا صعبة وسنلعب ب...
6,تنسيقية شباب الأحزاب تعد تقريرا بعنوان أكتوبر ...,أعدت تنسيقية شباب الأحزاب والسياسيين تقريرا تح...,تنسيقية شباب الأحزاب تعد تقريرا بعنوان أكتوبر ...
7,خلال اجتماعها المواد الغذائية ترفع تقريرا لالع...,رفعت شعبة المواد الغذائية بغرفة القاهرة تقريرا...,خلال اجتماعها المواد الغذائية ترفع تقريرا لالع...
8,السعودية تعطل الكمبيوتر الياباني في تصفيات مون...,خبرني حقق المنتخب السعودي فوزا ثمينا على نظيره...,السعودية تعطل الكمبيوتر الياباني في تصفيات مون...
9,اكرم يسرى جناح نادى الزمالك يحصل على لقب هداف ...,احتل لاعب نادى الزمالك اكرم يسرى صدارة هدافين ...,اكرم يسرى جناح نادى الزمالك يحصل على لقب هداف ...


### Saving Cleaned DF

In [ ]:
df_cleaned.to_csv("datasets/clean_data.csv")

## TF-IDF & Removing Arabic Stopwords

### Arabic Stopwords

In [76]:
# nltk.download("stopwords")
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
arb_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


### Applying TF-idf

In [97]:
documents = df_cleaned.Text.copy()

vectorizer = TfidfVectorizer(stop_words = arb_stopwords,
                             min_df = 0.01,
                             #ngram_range = (1,3),
                             #max_df = 7,
                            )

X_fit = vectorizer.fit_transform(documents)


C:\Users\ahmed\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


#### Converting to DF

In [98]:
#Converting to df
columns = vectorizer.get_feature_names()

        #using to dense to convert sparse to normal matrix
tf_idf = pd.DataFrame(X_fit.todense(), columns=columns, index=documents)
tf_idf.head(3)

,آثار,آخر,آخرين,آسيا,آفاق,آل,آلاف,آلام,آلية,أبحاث,...,يوجد,يوسف,يوفر,يوفنتوس,يوم,يوما,يومي,يوميا,يومين,يونايتد
Text,,,,,,,,,,,,,,,,,,,,,
سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف متابعة مظفر إسماعيل نفى لويس سواريز تعمده إرسال رسالة إلى رونالد كومان خلال لقاء أتلتيكو مدريد وضيفه برشلونة في الدوري الإسباني السبت وبعدما سجل سواريز هدف أتلتيكو الثاني أشار المهاجم المخضرم بيده وكأنه يمسك هاتفا ويتحدث إلى شخص ما وهو ما ربطه المتابعون بمكالمة كومان إلى سواريز في صيف التي أبلغه خلالها بأنه خرج من خططه وأن عليه الرحيل عن برشلونة والبحث عن فريق آخر وقال سواريز بعد اللقاء الذي ابتسم لأصحاب الأرض إن الرسالة التي أراد توجيهها لم تكن لكومان على الإطلاق وتابع بلهجة غامضة في تصريحات نشرتها صحيفة ماركا الرياضية الإسبانية كان ذلك للأشخاص الذين يعرفون أن لدي نفس الرقم وأنني ما زلت أستخدم نفس الهاتف إن كنت تريد أن تعرف كنت أمزح مع أطفالي أنني سأفعل ذلك كما علق سواريز على عدم احتفاله بإحراز هدف في شباك فريقه السابق موضحا كنت أعرف بالفعل أنني إذا سجلت فسأعتذر وأضاف لم أحتفل احتراما وتعاطفا لكوني من مشجعي برشلونة ولمسيرتي في برشلونة وللفترة التي يمر بها زملائي السابقون وهي صعبة ولجماهير برشلونة احترمت ذلك,0.0,0.086357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ماذا يفعل القرنفل على الريق خبرني القرنفل على الريق هي ذات الفوائد التي يجنيها الإنسان من خلال استخدامه للقرنفل في أي وقت وتتمثل فوائد القرنفل على الريق المحتملة في ما يأتي يخفض من خطر الإلتهابات فيساعد استخدام القرنفل في خفض خطر الإصابة بالالتهابات كما يتمتع بخصائص مضادة للأكسدة فإن القرنفل يحتوي على العديد من المواد والتي لها دور كمضادات للأكسدة وهذا يعني أنها تعمل على التقليل من مشكلة الإجهاد التأكسدي الناتجة عن تراكم الجذور الحرة في الجسم ويقلل من مشكلة تقرحات المعدة كما يزيد من عمل وكفاءة الكبد حيث تعمل المواد الموجودة فيه وخاصة مادة اليوجينول على التقليل من علامات الإصابة بالكبد الدهني وتشمع الكبد كما أنها تساعد في تحسين وظائف الكبد بشكل عام إضافة إلى أنه يحسن صحة الفم والأسنان ويقوي صحة الجهاز الهضمي إلى جانب دور القرنفل في التقليل من مشكلة تقرحات المعدة فإنه يعمل على تقوية صحة الجهاز الهضمي بشكل عام,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
لن تتوقعها حيلة ذكية تجعل ثمرة الأفوكادو تنضج خلال وقت قياسي متابعة علي معلا يحب الجميع تناول فاكهة الأفوكادو ذات المذاق الفريد والغنية بالمكونات الأساسية التي يحتاج إليها الجسم وعادة ما تتوفر في المتاجر غير ناضجة ولهذا من المفيد جدا تعلم طريقة تساعد على إنضاجها بسرعة في المنزل ومن أجل هذا يمكن اللجوء إلى حيلة الموز وفي هذه الحالة يجب وضع الحبة تحت بضع حبات من الموز لمدة أيام فهذه الثمار تصدر غاز الإيثيلين الذي يعمل على إنضاج الفواكه ويمكن ايضا تغليف حبات الأفوكادو بالورق لمدة أيام قليلة ومن الحيل المفيدة في هذا السياق أيضا أن يتم استخدام أعواد تنظيف الأسنان فهي عملية جدا ومفيدة ولهذا يجب غرز عود في ثمرة الأفوكادو من كل جانب وتركها حتى اليوم التالي حيث سيلاحظ أنها بدأت تنضج فعلا ويشار إلى أنه يمكن تناولها طازجة من دون المبالغة في ذلك كما يمكن إضافة شرحاتها إلى مختلف أنواع السلطات ومن المفيد الاعتماد عليها لتحضير طبق تحلية صحي وسريع وفي هذه الحالة ننصحك مثلا بإضافة القليل من العسل إلى قطع الأفوكادو,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## K-Means Clustering

### Elbow Method

> - It Computes in long hours, but it says that we have 6 clusters

In [ ]:
# #Drawing Elbow
# import matplotlib.pyplot as plt
# wcss = []
# for i in range(3,14):
#     kmeans = KMeans(n_clusters=i, init='random',max_iter=500,n_init=20,random_state=1)
#     kmeans.fit(X_fit)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(3,14),wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.savefig('elbow.png')
# plt.show()

### Applying Kmeans

In [79]:
true_k = 6
# model = KMeans(n_clusters=true_k, init='k-means++',random_state=60, max_iter=100, n_init=1)
model = KMeans(n_clusters=true_k, init='random', max_iter=700, n_init=20, random_state=1)
model.fit(X_fit)



KMeans(init='random', max_iter=700, n_clusters=6, n_init=20, random_state=1)

#### Saving the model

In [109]:
filename = "ar_kmeans.pkl"
pickle.dump(model, open(filename,"wb"))    #saving the model

### Getting top terms for each cluster

In [81]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()


for i in range(true_k):
    clust_words = []
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        clust_words.append(terms[ind])
    
    print(clust_words)
        

Top terms per cluster:
Cluster 0:
['الذهب', 'سعر', 'عيار', 'أسعار', 'جرام', 'دينارا', 'للأوقية', 'تعاملات', 'الدولار', 'اليوم']
Cluster 1:
['حالة', 'كورونا', 'إصابة', 'وفاة', 'جديدة', 'الصحة', 'تسجيل', 'إجمالي', 'بفيروس', 'عدد']
Cluster 2:
['يمكن', 'الجسم', 'الدم', 'تناول', 'القلب', 'الإصابة', 'يساعد', 'النوم', 'الوزن', 'بشكل']
Cluster 3:
['خلال', 'رئيس', 'المملكة', 'المتحدة', 'كورونا', 'العام', 'مجلس', 'الرئيس', 'بن', 'الخارجية']
Cluster 4:
['سعر', 'للشراء', 'للبيع', 'المركزي', 'الدولار', 'الدينار', 'بنك', 'صرف', 'متوسط', 'ليبيا']
Cluster 5:
['الدوري', 'فى', 'الفريق', 'منتخب', 'المباراة', 'مباراة', 'محمد', 'المنتخب', 'الزمالك', 'صلاح']


#### Results

> We can notice that topics are:
>> Cluster 0: Economy<br>
>> Cluster 1: Health<br>
>> Cluster 2: Health<br>
>> Cluster 3: Politics<br>
>> Cluster 4: Economy<br>
>> Cluster 5: Sports

### Adding Labels Columns to each Text

In [82]:
labels = dict()

labels[0] = 'اقتصاد'
labels[1] = 'صحة'
labels[2] = 'صحة'
labels[3] = 'سياسة'
labels[4] = 'اقتصاد'
labels[5] = 'رياضة'

df_clustered = df_cleaned.copy()

#Getting Results
df_clustered["Topic_AR"] = model.predict(X_fit)

#Replacing values
df_clustered = df_clustered.replace({"Topic_AR":labels})

In [83]:
df_clustered[['Text','Topic_AR']].head(15)

,Text,Topic_AR
0,سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف ...,رياضة
1,ماذا يفعل القرنفل على الريق خبرني القرنفل على ...,صحة
2,لن تتوقعها حيلة ذكية تجعل ثمرة الأفوكادو تنضج ...,صحة
3,السعودية تستضيف اليابان ضمن مباريات فى تصفيات ...,رياضة
4,انتهاء التصويت صناديق الاقتراع العام تغلق في ع...,سياسة
5,إنريكى مواجهة إسبانيا ضد إيطاليا صعبة وسنلعب ب...,رياضة
6,تنسيقية شباب الأحزاب تعد تقريرا بعنوان أكتوبر ...,سياسة
7,خلال اجتماعها المواد الغذائية ترفع تقريرا لالع...,سياسة
8,السعودية تعطل الكمبيوتر الياباني في تصفيات مون...,رياضة
9,اكرم يسرى جناح نادى الزمالك يحصل على لقب هداف ...,رياضة


## Total Pipeline to predict Label of new Text

In [161]:
input_text = '''
أسعار الذهب في البورصات العالمية شهدت ارتفاع بنسبة 1% تقريبًا وقت إغلاق السوق هذا الأسبوع، حيث عززت حالة عدم اليقين التي أثارها متغير فيروس كورونا أوميكرون وتراجع عوائد سندات الخزانة الأميركية من جاذبية المعدن الذي يعتبر ملاذًا آمنًا، وارتفع الذهب الفوري بنسبة 0.9% إلى 1785.29 دولارًا للأونصة، واستقرت عقود الذهب الأمريكية الآجلة على ارتفاع بنسبة 1.2% إلى 1783.90 دولار.

وظلت المعنويات في الأسواق المالية ضعيفة، حيث انخفض مؤشر ناسداك بما يزيد عن 2% ، مع تباين بيانات الوظائف الأميركية والمخاوف بشأن أوميكرون، كما دعم أسعار الذهب، انخفاض عائد السندات الأميركية لأجل 10 سنوات إلى أقل من 1.4٪ للمرة الأولى منذ سبتمبر، مما قلل من تكلفة الفرصة البديلة لامتلاك الذهب بدون فوائد.
 '''

In [162]:
text = input_text

# Step 1: Clean text
cleaned = clean(text)
# print(cleaned)


# Step 2: Vecotrizer (Tf-idf)
cleaned = [cleaned]
vectorized = vectorizer.transform(cleaned)


# Step 3: Predicting
predicted = model.predict(vectorized)
print(predicted)


# Step 4: Converting to Category
print('Category of this Text is : {}'.format(labels[predicted[0]]))

[0]
Category of this Text is : اقتصاد


<hr>
<hr>
<hr>
<hr>
<hr><hr>

# Summarization

## Working on raw text

In [163]:
text = input_text
text

'\nأسعار الذهب في البورصات العالمية شهدت ارتفاع بنسبة 1% تقريبًا وقت إغلاق السوق هذا الأسبوع، حيث عززت حالة عدم اليقين التي أثارها متغير فيروس كورونا أوميكرون وتراجع عوائد سندات الخزانة الأميركية من جاذبية المعدن الذي يعتبر ملاذًا آمنًا، وارتفع الذهب الفوري بنسبة 0.9% إلى 1785.29 دولارًا للأونصة، واستقرت عقود الذهب الأمريكية الآجلة على ارتفاع بنسبة 1.2% إلى 1783.90 دولار.\n\nوظلت المعنويات في الأسواق المالية ضعيفة، حيث انخفض مؤشر ناسداك بما يزيد عن 2% ، مع تباين بيانات الوظائف الأميركية والمخاوف بشأن أوميكرون، كما دعم أسعار الذهب، انخفاض عائد السندات الأميركية لأجل 10 سنوات إلى أقل من 1.4٪ للمرة الأولى منذ سبتمبر، مما قلل من تكلفة الفرصة البديلة لامتلاك الذهب بدون فوائد.\n '

### Tf-idf Method to Summarize

#### Getting words Importance

In [173]:
word_frequencies = {}  

for word in nltk.word_tokenize(text):  
    if word not in arb_stopwords:
        if word not in punctuation:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
                
maximum_frequncy = max(word_frequencies.values())
for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

In [174]:
word_frequencies

{'أسعار': 0.5,
 'الذهب': 1.0,
 'البورصات': 0.25,
 'العالمية': 0.25,
 'شهدت': 0.25,
 'ارتفاع': 0.5,
 'بنسبة': 0.75,
 '1': 0.25,
 'تقريبًا': 0.25,
 'وقت': 0.25,
 'إغلاق': 0.25,
 'السوق': 0.25,
 'الأسبوع،': 0.25,
 'عززت': 0.25,
 'حالة': 0.25,
 'عدم': 0.25,
 'اليقين': 0.25,
 'أثارها': 0.25,
 'متغير': 0.25,
 'فيروس': 0.25,
 'كورونا': 0.25,
 'أوميكرون': 0.25,
 'وتراجع': 0.25,
 'عوائد': 0.25,
 'سندات': 0.25,
 'الخزانة': 0.25,
 'الأميركية': 0.75,
 'جاذبية': 0.25,
 'المعدن': 0.25,
 'يعتبر': 0.25,
 'ملاذًا': 0.25,
 'آمنًا،': 0.25,
 'وارتفع': 0.25,
 'الفوري': 0.25,
 '0.9': 0.25,
 '1785.29': 0.25,
 'دولارًا': 0.25,
 'للأونصة،': 0.25,
 'واستقرت': 0.25,
 'عقود': 0.25,
 'الأمريكية': 0.25,
 'الآجلة': 0.25,
 '1.2': 0.25,
 '1783.90': 0.25,
 'وظلت': 0.25,
 'المعنويات': 0.25,
 'الأسواق': 0.25,
 'المالية': 0.25,
 'ضعيفة،': 0.25,
 'انخفض': 0.25,
 'مؤشر': 0.25,
 'ناسداك': 0.25,
 'يزيد': 0.25,
 '2': 0.25,
 '،': 0.25,
 'تباين': 0.25,
 'بيانات': 0.25,
 'الوظائف': 0.25,
 'والمخاوف': 0.25,
 'بشأن': 0.25,
 'أوميكر

#### Getting Sentences Importance

In [203]:
sentence_scores = {} 

In [204]:
sent_stop_words = r'[،!\-\?\,]|\. '

#Taking : ! ? - , ،
#Taking (. that comes after it a space) because we have digits like (1.4, 2.5, etc)

In [205]:

sentence_list = re.split(sent_stop_words,text)
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

In [206]:
sentence_scores

{'\nأسعار الذهب في البورصات العالمية شهدت ارتفاع بنسبة 1% تقريبًا وقت إغلاق السوق هذا الأسبوع': 4.75,
 ' حيث عززت حالة عدم اليقين التي أثارها متغير فيروس كورونا أوميكرون وتراجع عوائد سندات الخزانة الأميركية من جاذبية المعدن الذي يعتبر ملاذًا آمنًا': 5.0,
 ' وارتفع الذهب الفوري بنسبة 0.9% إلى 1785.29 دولارًا للأونصة': 3.0,
 ' واستقرت عقود الذهب الأمريكية الآجلة على ارتفاع بنسبة 1.2% إلى 1783.90 دولار.\n\nوظلت المعنويات في الأسواق المالية ضعيفة': 4.75,
 ' حيث انخفض مؤشر ناسداك بما يزيد عن 2% ': 1.25,
 ' مع تباين بيانات الوظائف الأميركية والمخاوف بشأن أوميكرون': 2.25,
 ' كما دعم أسعار الذهب': 1.75,
 ' انخفاض عائد السندات الأميركية لأجل 10 سنوات إلى أقل من 1.4٪ للمرة الأولى منذ سبتمبر': 3.0,
 ' مما قلل من تكلفة الفرصة البديلة لامتلاك الذهب بدون فوائد.\n ': 2.75}

#### Joining Sentences according to their importance

In [207]:
number_of_sentence = 3
summary_sentences = heapq.nlargest(number_of_sentence, sentence_scores, key=sentence_scores.get)
summary_sentences
# summary = ' '.join(summary_sentences) 

# summary

[' حيث عززت حالة عدم اليقين التي أثارها متغير فيروس كورونا أوميكرون وتراجع عوائد سندات الخزانة الأميركية من جاذبية المعدن الذي يعتبر ملاذًا آمنًا',
 '\nأسعار الذهب في البورصات العالمية شهدت ارتفاع بنسبة 1% تقريبًا وقت إغلاق السوق هذا الأسبوع',
 ' واستقرت عقود الذهب الأمريكية الآجلة على ارتفاع بنسبة 1.2% إلى 1783.90 دولار.\n\nوظلت المعنويات في الأسواق المالية ضعيفة']

#### ** Trying taking always sentence number 1 -> By increasing its weight

In [211]:
number_of_sentence = 3
sentence_scores[next(iter(sentence_scores))] += 100
summary_sentences = heapq.nlargest(number_of_sentence, sentence_scores, key=sentence_scores.get)

for sent in summary_sentences:
    clean_regex = r'(\\\w)+'   #Removing \n \t
    sent = re.sub(clean_regex,'',sent)
    print(sent)


أسعار الذهب في البورصات العالمية شهدت ارتفاع بنسبة 1% تقريبًا وقت إغلاق السوق هذا الأسبوع
 حيث عززت حالة عدم اليقين التي أثارها متغير فيروس كورونا أوميكرون وتراجع عوائد سندات الخزانة الأميركية من جاذبية المعدن الذي يعتبر ملاذًا آمنًا
 واستقرت عقود الذهب الأمريكية الآجلة على ارتفاع بنسبة 1.2% إلى 1783.90 دولار.

وظلت المعنويات في الأسواق المالية ضعيفة
